<h2>This script is for internal developer testing.</h2>

In [ ]:
import wfdb

In [ ]:
# Testing rdsamp multiple samples per frame (and fmt 12)
sig, fields=wfdb.srdsamp('sampledata/03700181')
print(sig[:20,:])
print("\n\n", sig.shape, "\n\n")
print(fields)

print("\n\n\n")
print(sig[-20:,:])

In [ ]:
# Testing rdsamp format 80
sig, fields=wfdb.srdsamp('sampledata/3000003_0003')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
sig, fields=wfdb.srdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# For writing test annotation files based on first 3 annotations of 12726.anI

f=open('tmp.fake', 'wb')

indata= np.array([0, 236,   1,   0, 200,  84,   0,  88, 255, 251,  21, 252,  73, 110, 105, 116,
                  105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
                  112,   0,   0, 236,   0,   0,  67,  50,   0,  88,  21, 252,  67, 111, 110,  99,
108, 117, 100, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
112,   0,   0, 236,   0,   0, 114, 183,   0,  88,  23, 252,  73, 110, 105, 116,
105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 100,
111, 119, 110,   0, 0 ,0 ]).astype('u1')


print(indata)

indata.tofile(f)


f.close()


In [ ]:
# Trying to fix byte pointer for reading skewed signals. 

from wfdb import readsignal
import numpy as np
#sig, fields=readsignal.rdsamp('sampledata/03700181', channels=[0,2], sampfrom=1000, sampto=16000)
#sig=np.round(sig, decimals=8)
#targetsig=np.genfromtxt('tests/targetoutputdata/target10')

sig, fields=readsignal.rdsamp('sampledata/03700181', sampfrom=1000, sampto=16000, physical=0) # fs=125. -f 8 -t 128

print("\nsig: ", sig)
print('\n\n')
print("End of sig: ", sig[-10:,:])


# nbytesread = 15000 frames * 6 samples/frame * 1.5 bytes/sample = 135000

# Try to octal dump the right section of the file. After 16000 frames x (6 samples/frame)  = 96000 samples * (1.5 bytes per sample)
# = 144000 bytes skip. 



# od -t u1 03700181.dat -j 144000 -N 30 

# Result: 
# 236 255 236   2 240 236 163  27  61 236 255 236 236  15   2 147
#  27  40 236 255 236 236 255 236 130  27  19   2 240 236




In [ ]:
import wfdb
ann = wfdb.rdann('/home/chen/Downloads/chb01_03.edf','seizures')

In [ ]:
ann.__dict__

In [ ]:
# Testing wrann
import wfdb
from IPython.display import display

annsamp = [20, 40, 60, 80, 100, 800, 1200 , 3000, 3001, 8000, 100000]
#annsamp = [20, 40, 60, 80, 100, 800, 1200]
#annsamp = [20, 60, 2**31] # 2**31+100 is too large

nsamp = len(annsamp)

anntype = ['N', 'N', 'N', '3', 'N', '2', 'N', '1', 'N', 'N', 'g']
sub = [1,0,0,0,5,3,2,5,0,0,0]
chan = [2,2,3,2,9,2,2,2,2,2,2]
num = [1,2,3,4,5,3,2,5,11,22,22]
aux = [None, None, "good", "good", "good", "good", "good", "good", None, None, None]
fs = 100
fs = None

a = wfdb.Annotation(recordname = 'firstann', annotator = 'atr', 
                    annsamp = annsamp, anntype = anntype, aux = aux, 
                    subtype = sub, chan = chan, num = num, fs = fs)
a.type2aux()
display(a.__dict__)
a.wrann()

b = wfdb.rdann('firstann', 'atr')
b.__dict__


# print([sd & 255, sd & 768, (sd & 768) >> 8])

In [ ]:
import numpy as np
inttypes = [int, np.int64, np.int32]

print(inttypes)
display(inttypes)

In [6]:
class circle():
    
    def __init__(self):
        self.shape = 'round'
        
    def circle2square(self):
        
        return square()
    
        
class square():
    
    def __init__(self):
        self.shape = 'rectangular'
        

def printandswitch(circle):
    print('hi')
    circle = circle.circle2square()
    print(type(circle))
    
myshape = circle()
printandswitch(myshape)
print(type(myshape))
myshape = myshape.circle2square()
print(type(myshape))
        
        

hi
<class '__main__.square'>
<class '__main__.circle'>
<class '__main__.square'>


In [ ]:
import numpy as np
from . import _headers
import matplotlib.pyplot as plt


def plotrec(record=None, signals = None, fields = None, title=None, timeaxis='samples'): 
    
    # Figure out which argument to plot
    if record is None:
        if signals is None:
            sys.exit('Either record or signals must be present. No input signal to plot.')
        # Use the signals array
    else:
        # Use the record object
        if signals is None:
            # If it is a MultiRecord, convert it into single
            if type(record) == MultiRecord:
                record = record.multi_to_single()
            
            # Need to ensure p_signals is present
            if record.p_signals is None:
                sys.exit('The p_signals field must be present in the record object to plot')
                
            signals = record.p_signals
            fields = {}
            for field in ['fs', 'signame']:
                fields[field] = getattr(record, field)
        # Both are present
        else:
            sys.exit("Only one of 'record' or 'signals' can be input. Cannot plot both items.")
            
    # Determine x axis resolution
    allowedtimes = ['samples, seconds, minutes, hours']
    if timeaxis not in allowedtimes:
        print("The 'timeaxis' field must be one of the following: ", timeaxis)
        sys.exit()
        
    # Check the validity of the signals and fields
    checkplotitems(signals, fields)
        
    # Check that the fs field is present for time plots
    if timeaxis != 'samples':
        if type(fields['fs']) not in _headers.floattypes:
            sys.exit("The 'fs' field must be a number")
     
    
    
    if timeaxis == 'samples':
        t = np.array(range(signals.shape[0]))
    
    
    if (fields["fs"] is None) or (plottime is not True): # x axis is index
        plottime=False
        t=np.array(range(0,sig.shape[0]))
        
    else: # x axis is time
        t=np.array(range(0,sig.shape[0]))/fields["fs"]
        if annsamp!=[]:
            annplott=annsamp/fields["fs"]
        
    f1=plt.figure()
    for ch in range(0, sig.shape[1]):
        plt.subplot(100*sig.shape[1]+11+ch)
        plt.plot(t, sig[:,ch]) # Plot signal channel
        if (annsamp!=[]) & (ch in annch): # If there are annotations to plot and the channel is specified
            plt.plot(annplott, sig[annsamp, ch], 'r+') # Plot annotations
        
        if (title!=[])&(ch==0):
            plt.title(title)
            
        # Axis Labels
        if plottime:
            plt.xlabel('time/s')
        else:
            plt.xlabel('index/sample')
        if fields["signame"][ch]:
            chanlabel=fields["signame"][ch]
        else:
            chanlabel='channel'
        if fields["units"][ch]:
            unitlabel=fields["units"][ch]
        else:
            unitlabel='NU'
        plt.ylabel(chanlabel+"/"+unitlabel)
        
    plt.show(f1)

    
    
def checkplotitems(signals, fields):
    
    # signals
    if type(signals) != np.ndarray or signals.ndim != 2
        sys.exit("The plotted signal must be a 2d numpy array")
    
    # fs
    if type(fields['fs']) not in _headers.floattypes:
            sys.exit("The 'fs' field must be a number")
    
    # signame
    if fields['signame'] is None:
        fields['signame'] = 
    
    return fields

In [7]:
import wfdb

x = wfdb.Record()


In [2]:
type(5.4) == float

True